In [1]:
import requests, time, json 
from requests.auth import HTTPBasicAuth
import json
import pandas as pd
from pandas.io.json import json_normalize
from authlib.jose import jwt
import io
import os
from os import chdir

#module 1 for AppFollow Connect 
#save apps_id and authkey in a different package for many other types of API url requests
#request service get on the AppFollow Api for the list of games 
response = requests.get('http://api.appfollow.io/apps/app?apps_id=48190',auth=HTTPBasicAuth('2DkkHPu5iAHzWBDCGaWz','')) #authkey can be different in future

#storing the json output(dict) in a variable data
data=response.json()
#module 1 end for AppFollow


#module 3 AppFollow data process
#converting the extracted data into a DataFrame
#df1=pd.DataFrame(data['apps_app'])
df1=pd.json_normalize(data['apps_app'])

#droping the columns using the names
#we can use df1.columns to find the names of the columns in an array
df1.drop(['count_whatsnew','watch_url','has_reply_integration','is_favorite','app.version','app.kind','app.icon','app.genre','app.has_iap','app.artist_name','app.id','app.lang','app.url','app.type'], axis=1, inplace=True)

#renaming the app.ext_id column to ext_id to be able to join
df1 = df1.rename(columns = {'app.ext_id': 'ext_id'}, inplace = False)
#saving the Extracted data from appfollow in the data folder created 
df1.to_csv('D:/Data/appfollow.csv')
#module 3 end

###Module 2 of Authorisation
#Accessing AppStore Connect API requires JWT token which inturns requires Key_id,issuer_id,Private_key 
KEY_ID = "9G6CVN97LT"
ISSUER_ID = "69a6de7b-3f5f-47e3-e053-5b8c7c11a4d1"
EXPIRATION_TIME = int(round(time.time()))
PATH_TO_KEY = 'C:/Users/Lenovo/Downloads/AuthKey_9G6CVN97LT.p8'
with open(PATH_TO_KEY, 'r') as f:
    PRIVATE_KEY = f.read()
    
header = {
    "alg": "ES256",
    "kid": KEY_ID,
    "typ": "JWT"
}

payload = {
    "iss": ISSUER_ID,
    "exp": EXPIRATION_TIME,
    "aud": "appstoreconnect-v1"
}

#JWT token
token = jwt.encode(header, payload, PRIVATE_KEY)

JWT = 'Bearer ' + token.decode()

URL = 'https://api.appstoreconnect.apple.com/v1/apps' #The API to get the data of games on AppStore Connect
HEAD = {'Authorization': JWT}
###Module 2 End Authorization


###modeule 4 AppStore Connect Data processing
#request get service on the URL using the parameter and JWT
r = requests.get(URL, params={'limit': 200}, headers=HEAD) #params limit 200 apps for AppStore(might need to change in future for more apps)
#storing the json output in a variable name x
x=r.json()
df2=pd.DataFrame(x['data'])
df2.drop(df2.iloc[:, 6:40], inplace = True, axis = 1)
#remaning the id column as ext_id to be able to join
df2 = df2.rename(columns = {'id': 'ext_id'}, inplace = False)
#saving the extracted data into the data folder created in D: drive
df2.to_csv('D:/Data/appstore.csv')
###Module 4 End


#module 5 Extracted data processing and creating viewpoint(status of AppStore Apps Integration)
#importing the two extracted data csv file from the same location
df3 = pd.read_csv (r'D:/Data/appfollow.csv')
df4 = pd.read_csv(r'D:/Data/appstore.csv')
consolidated.shape
#join the two extracted data file(csv) using the left join AppStore on AppFollow to find the integrated and not integrated Apps
consolidated = pd.merge(df4, df3, on='ext_id', how='left')
consolidated.shape #should show all columns from AppStore
#filter the test and prototype bundle id rows
#filter = consolidated['attributes.bundleId'].str.contains('test')
#consolidated=consolidated[~filter]
#filter = consolidated['attributes.bundleId'].str.contains('prototype')
#consolidated=consolidated[~filter]
#Its better to reset about index after every filter, eg:- reset_index
status = []
for row in consolidated['store']:
    if row=="itunes" :
            status.append('Integrated')
    else :
            status.append('Not Integrated')           
# Creating a column named status
consolidated['status'] = status
consolidated.to_csv('D:/Data/combined.csv')
#module 5 end